In [1]:
from transformers import AutoTokenizer
import torch
import math

class QFNNPolarTokenBank:
    def __init__(self, model_name="gpt2", device="cuda" if torch.cuda.is_available() else "cpu"):
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokens = list(self.tokenizer.get_vocab().keys())
        self.golden_ratio = (1 + math.sqrt(5)) / 2
        self.token_polar_map = self.build_polar_token_map()
    
    def build_polar_token_map(self):
        N = len(self.tokens)
        i = torch.arange(N, dtype=torch.float32, device=self.device)
        
        # Phase angle with golden ratio
        θ = (i * self.golden_ratio * 2 * torch.pi) % (2 * torch.pi)
        
        # Radial position (√(n/N)) for natural quantum embedding
        r = 0.3 + 0.7 * torch.sqrt((i + 1.0) / N)

        token_polar_map = {}
        for idx, token in enumerate(self.tokens):
            token_polar_map[token] = (r[idx].item(), θ[idx].item())
        
        return token_polar_map
    
    def get_token_coordinates(self, token):
        return self.token_polar_map.get(token, None)


In [ ]:
from qfnn_polar_token_bank import QFNNPolarTokenBank

token_bank = QFNNPolarTokenBank()
coords = token_bank.get_token_coordinates('the')
print(coords)  # (r, θ) for 'the'
